#Ансамбли моделей машинного обучения.
Цель лабораторной работы: изучение ансамблей моделей машинного обучения.

##Требования к отчету:
Отчет по лабораторной работе должен содержать:

1. титульный лист;
2. описание задания;
3. текст программы;
4. экранные формы с примерами выполнения программы.

##Задание:
1. Выберите набор данных (датасет) для решения задачи классификации или регресии.

2. В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков.

3. С использованием метода train_test_split разделите выборку на обучающую и тестовую.

4. Обучите следующие ансамблевые модели:

 - одну из моделей группы бэггинга (бэггинг или случайный лес или сверхслучайные деревья);
 - одну из моделей группы бустинга;
 - одну из моделей группы стекинга.

5. Оцените качество моделей с помощью одной из подходящих для задачи метрик. Сравните качество полученных моделей.


### Шаг 1: Предобработка данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv('nasa.csv')

# Проверяем наличие пропусков и, в случае необходимости, заполняем или удаляем пропущенные значения
df.isnull().sum()
df.fillna(0, inplace=True)

# Преобразование строк с датами в datetime
df['Close Approach Date'] = pd.to_datetime(df['Close Approach Date'])
df['Orbit Determination Date'] = pd.to_datetime(df['Orbit Determination Date'])

# Перевод дат в количество дней с начала эпохи
df['Close Approach Date(days)'] = (df['Close Approach Date'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1D')
df['Orbit Determination Date(days)'] = (df['Orbit Determination Date'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1D')

df = df.drop(['Close Approach Date', 'Orbit Determination Date'], axis=1)
df = df.drop(['Equinox'], axis = 1)

# Кодирование категориальных признаков если они есть
# Пример кодирования для 'Orbiting Body' если в нем есть пропуски или не числовые значения
df = pd.get_dummies(df, columns=['Orbiting Body'])

###Шаг 2: Разделяем на обучающую и тестирующую выборку

In [ ]:
X = df.drop('Hazardous', axis=1)
y = df['Hazardous']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


###Шаг 3: Обучим модели

- Беттинг ( случайный лес )

In [ ]:
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

- Бустинг ( XGBoost )

In [ ]:
xgb_clf = XGBClassifier(random_state=42)
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

- Стекинг

In [ ]:
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
]
stack_clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
stack_clf.fit(X_train, y_train)


StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(n_estimators=10,
                                                       random_state=42)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='logloss',
                                              feature_types=None, gamma=None,
                                              grow_pol...
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=42, ...))],
                   final_estimator=LogisticRegression())

###Шаг 4: Оценим качество моделей с помощью подходящий метрик


In [ ]:
models = [rf_clf, xgb_clf, stack_clf]
model_names = ['Random Forest', 'XGBoost', 'Stacking']
accuracy_array = []
for model, name in zip(models, model_names):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_array.append(accuracy)
    print(f'{name} Accuracy: {accuracy:.4f}')


Random Forest Accuracy: 0.9950
XGBoost Accuracy: 0.9943
Stacking Accuracy: 0.9943


###Шаг 5: Сравним качество полученных моделей.

In [ ]:
# Сравнение моделей
best_accuracy = max(accuracy_array)
for i in range (len(model_names)):
  if best_accuracy == accuracy_array[i]:
    print(f'Best model: {model_names[i]}\n Accuracy: {accuracy_array[i]}')


Best model: Random Forest
 Accuracy: 0.9950248756218906
